In [1]:
from dash import Dash, html, dcc, Output, Input
import dash_cytoscape as cyto
import dash_bootstrap_components as dbc

# Tree data, needs to come from the analysts input and the automatically calculated complements
base_nodes = [
    {'id': 'root', 'label': 'All Cases', 'count': 31509, 'percentage': 100.00, 'avg_duration': '21d 21h 34m', 'avg_amount': 16233.74},
    {'id': 'high_freq', 'label': 'High-Frequency Variants (≥5)', 'count': 13960, 'percentage': 44.3, 'avg_duration': '22d 10h 38m', 'avg_amount': 15270.71},
    {'id': 'high_freq_ge2', 'label': '≥2 O_Create Offer', 'count': 1434, 'percentage': 10.27, 'avg_duration': '29d 14h 42m', 'avg_amount': 15760.46},
    {'id': 'high_freq_ge2_pending', 'label': 'With A_Pending', 'count': 307, 'percentage': 21.41, 'avg_duration': '16d 6h 7m', 'avg_amount': 14516.43},
    {'id': 'high_freq_ge2_not_pending', 'label': 'Without A_Pending', 'count': 1127, 'percentage': 78.59, 'avg_duration': '33d 6h 1m', 'avg_amount': 16099.33},
    {'id': 'high_freq_lt2', 'label': '<2 O_Create Offer', 'count': 12526, 'percentage': 89.73, 'avg_duration': '21d 14h 56m', 'avg_amount': 15214.64},
    {'id': 'high_freq_lt2_pending', 'label': 'With A_Pending', 'count': 5019, 'percentage': 40.07, 'avg_duration': '13d 15h 50m', 'avg_amount': 15061.33},
    {'id': 'high_freq_lt2_not_pending', 'label': 'Without A_Pending', 'count': 7507, 'percentage': 59.93, 'avg_duration': '26d 22h 41m', 'avg_amount': 15317.15},
    {'id': 'low_freq', 'label': 'Low-Frequency Variants (<5)', 'count': 17549, 'percentage': 55.7, 'avg_duration': '21d 11h 11m', 'avg_amount': 16999.82},
    {'id': 'low_freq_ge2', 'label': '≥2 O_Create Offer', 'count': 7125, 'percentage': 40.6, 'avg_duration': '25d 9h 16m', 'avg_amount': 17708.45},
    {'id': 'low_freq_ge2_pending', 'label': 'With A_Pending', 'count': 4743, 'percentage': 66.57, 'avg_duration': '23d 11h 43m', 'avg_amount': 18058.93},
    {'id': 'low_freq_ge2_not_pending', 'label': 'Without A_Pending', 'count': 2382, 'percentage': 33.43, 'avg_duration': '29d 3h 58m', 'avg_amount': 17010.59},
    {'id': 'low_freq_lt2', 'label': '<2 O_Create Offer', 'count': 10424, 'percentage': 59.4, 'avg_duration': '18d 18h 52m', 'avg_amount': 16515.46},
    {'id': 'low_freq_lt2_pending', 'label': 'With A_Pending', 'count': 7159, 'percentage': 68.68, 'avg_duration': '17d 19h 16m', 'avg_amount': 16638.53},
    {'id': 'low_freq_lt2_not_pending', 'label': 'Without A_Pending', 'count': 3265, 'percentage': 31.32, 'avg_duration': '20d 22h 38m', 'avg_amount': 16245.62},
]

base_edges = [
    {'source': 'root', 'target': 'high_freq'},
    {'source': 'high_freq', 'target': 'high_freq_ge2'},
    {'source': 'high_freq_ge2', 'target': 'high_freq_ge2_pending'},
    {'source': 'high_freq_ge2', 'target': 'high_freq_ge2_not_pending'},
    {'source': 'high_freq', 'target': 'high_freq_lt2'},
    {'source': 'high_freq_lt2', 'target': 'high_freq_lt2_pending'},
    {'source': 'high_freq_lt2', 'target': 'high_freq_lt2_not_pending'},
    {'source': 'root', 'target': 'low_freq'},
    {'source': 'low_freq', 'target': 'low_freq_ge2'},
    {'source': 'low_freq_ge2', 'target': 'low_freq_ge2_pending'},
    {'source': 'low_freq_ge2', 'target': 'low_freq_ge2_not_pending'},
    {'source': 'low_freq', 'target': 'low_freq_lt2'},
    {'source': 'low_freq_lt2', 'target': 'low_freq_lt2_pending'},
    {'source': 'low_freq_lt2', 'target': 'low_freq_lt2_not_pending'},
]

base_path_nodes = ['root', 'high_freq', 'high_freq_lt2', 'high_freq_lt2_pending']

# Helper functions
def build_selected_path(frequency, offers, pending):
    path = ['root']
    if frequency == 'high':
        path.append('high_freq')
        if offers == 'lt2':
            path.append('high_freq_lt2')
            path.append('high_freq_lt2_pending' if pending == 'pending' else 'high_freq_lt2_not_pending')
        else:
            path.append('high_freq_ge2')
            path.append('high_freq_ge2_pending' if pending == 'pending' else 'high_freq_ge2_not_pending')
    else:
        path.append('low_freq')
        if offers == 'lt2':
            path.append('low_freq_lt2')
            path.append('low_freq_lt2_pending' if pending == 'pending' else 'low_freq_lt2_not_pending')
        else:
            path.append('low_freq_ge2')
            path.append('low_freq_ge2_pending' if pending == 'pending' else 'low_freq_ge2_not_pending')
    return path

def build_elements(measure='percentage', selected_path=None):
    elements = []
    base_nodes_set = set(base_path_nodes)
    compare_nodes_set = set(selected_path) if selected_path else set()

    for node in base_nodes:
        if measure == 'percentage':
            label = f"{node['label']}\n{node['count']} cases ({node['percentage']:.2f}%)"
        elif measure == 'avg_duration':
            label = f"{node['label']}\nAvg Duration: {node['avg_duration']}"
        elif measure == 'avg_amount':
            label = f"{node['label']}\n{node['avg_amount']:,.2f} EUR"
        else:
            label = node['label']

        if node['id'] in base_nodes_set:
            classes = 'base-path'
        elif node['id'] in compare_nodes_set:
            classes = 'compare-path'
        else:
            classes = ''

        elements.append({'data': {'id': node['id'], 'label': label}, 'classes': classes})

    for edge in base_edges:
        if edge['source'] in base_nodes_set and edge['target'] in base_nodes_set:
            classes = 'base-path'
        elif edge['source'] in compare_nodes_set and edge['target'] in compare_nodes_set:
            classes = 'compare-path'
        else:
            classes = ''

        elements.append({'data': {'source': edge['source'], 'target': edge['target']}, 'classes': classes})

    return elements

def path_info(path, measure):
    node_map = {node['id']: node for node in base_nodes}
    info = []
    for node_id in path:
        node = node_map[node_id]
        label = node['label']
        if measure == 'percentage':
            info.append(f"{label} ({node['count']} cases, {node['percentage']:.2f}%)")
        elif measure == 'avg_duration':
            info.append(f"{label} (Avg Duration: {node['avg_duration']})")
        elif measure == 'avg_amount':
            info.append(f"{label} (Avg Amount: {node['avg_amount']:,.2f} EUR)")
    return info

# App setup
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Div([
        html.H3("Analyst's path:", style={'margin-bottom': '5px'}),
        html.H4("All Cases → High Frequency → <2 Offers → Pending", style={'color': '#5DADE2', 'margin-bottom': '5px'}),

        html.H3("Alternative path:", style={'margin-bottom': '20px'}),

        html.Div([
            html.Div([
                html.Label('Frequency:'),
                dcc.RadioItems(
                    id='frequency-toggle',
                    options=[{'label': 'High', 'value': 'high'}, {'label': 'Low', 'value': 'low'}],
                    value='high',
                    inline=True,
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                )
            ]),
            html.Div([
                html.Label('Offers:'),
                dcc.RadioItems(
                    id='offers-toggle',
                    options=[{'label': '<2', 'value': 'lt2'}, {'label': '≥2', 'value': 'ge2'}],
                    value='lt2',
                    inline=True,
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                )
            ]),
            html.Div([
                html.Label('Pending:'),
                dcc.RadioItems(
                    id='pending-toggle',
                    options=[{'label': 'Pending', 'value': 'pending'}, {'label': 'Not Pending', 'value': 'not_pending'}],
                    value='pending',
                    inline=True,
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                )
            ])
        ], style={'display': 'flex', 'gap': '40px', 'margin': '20px'})
    ]),

    dcc.Dropdown(
        id='measure-dropdown',
        options=[
            {'label': 'Percentage of Cases', 'value': 'percentage'},
            {'label': 'Average Duration', 'value': 'avg_duration'},
            {'label': 'Average Requested Amount', 'value': 'avg_amount'}
        ],
        value='percentage',
        clearable=False,
        style={'width': '300px', 'margin': '20px'}
    ),

    cyto.Cytoscape(
        id='cytoscape-tree',
        layout={'name': 'breadthfirst', 'directed': True, 'spacingFactor': 2, 'roots': ['root'], 'padding': 100},
        style={'width': '100%', 'height': '1000px'},
        elements=build_elements(),
        stylesheet=[
            {'selector': 'node', 'style': {'label': 'data(label)', 'text-wrap': 'wrap', 'background-color': '#E5E8E8',
                                           'width': '340px', 'height': '120px', 'shape': 'rectangle',
                                           'text-valign': 'center', 'text-halign': 'center',
                                           'font-size': '24px', 'border-color': '#B3B6B7', 'border-width': 5}},
            {'selector': 'edge', 'style': {'line-color': '#ccc', 'width': 5}},
            {'selector': '.base-path', 'style': {'border-color': '#5DADE2', 'border-width': 5}},
            {'selector': '.compare-path', 'style': {'border-color': '#F5B041', 'border-width': 5}},
            {'selector': 'edge.base-path', 'style': {'line-color': '#5DADE2', 'width': 5}},
            {'selector': 'edge.compare-path', 'style': {'line-color': '#F5B041', 'width': 5}}
        ]
    ),

    html.H3("Comparison of Paths", style={'margin-top': '40px'}),
    html.Div(id='path-comparison')
])

# Callback for updating
@app.callback(
    Output('cytoscape-tree', 'elements'),
    Output('path-comparison', 'children'),
    Input('measure-dropdown', 'value'),
    Input('frequency-toggle', 'value'),
    Input('offers-toggle', 'value'),
    Input('pending-toggle', 'value')
)
def update_tree(measure, frequency, offers, pending):
    selected_path = build_selected_path(frequency, offers, pending)
    base_info = path_info(base_path_nodes, measure)
    selected_info = path_info(selected_path, measure)

    comparison_panel = html.Div([
        html.H4("Path Comparison", style={'margin-top': '30px'}),
        html.Div([
            html.Div([
                html.H5("Analyst's Path (Blue)", style={'color': '#5DADE2'}),
                html.Ul([html.Li(step) for step in base_info])
            ], style={'width': '45%', 'display': 'inline-block', 'vertical-align': 'top', 'padding': '10px'}),
            html.Div([
                html.H5("Alternative Path (Orange)", style={'color': '#F5B041'}),
                html.Ul([html.Li(step) for step in selected_info])
            ], style={'width': '45%', 'display': 'inline-block', 'vertical-align': 'top', 'padding': '10px'})
        ], style={'display': 'flex', 'justify-content': 'space-around'})
    ])

    return build_elements(measure, selected_path), comparison_panel

# Run
if __name__ == '__main__':
    app.run(debug=True)
